In [7]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Predict startup acquisition/ipo valuation amount based on funding history, individual background, industry, and other factors.

In [2]:
#loading data
acqs = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/acquisitions.csv')
degs = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/degrees.csv')
rounds = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/funding_rounds.csv')
funds = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/funds.csv')
invs = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/investments.csv')
ipos = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/ipos.csv')
objs = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/objects.csv')
ofcs = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/offices.csv')
ppl = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/people.csv')
relns = pd.read_csv('/Users/kirk.hachigian/Desktop/21SpringClasses/3120/Project/relationships.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
#response variable for each company = ipo valuation or acquisition price
Y1 = objs.merge(acqs,left_on='id',right_on='acquired_object_id')
Y1 = Y1[['id_x','name','price_amount']]
#Y1 = Y1[Y1['price_amount']!=0.0]
Y1.columns = ['id','name','valuation']

Y2 = objs.merge(ipos,left_on='id',right_on='object_id')
Y2 = Y2[['id_x','name','valuation_amount']]
Y2 = Y2[Y2['valuation_amount']!=0.0]
Y2.columns = ['id','name','valuation']

Y = Y1.append(Y2)
Y.reset_index(inplace=True, drop=True)
Y

,id,name,valuation
0,c:10,Flektor,2.000000e+07
1,c:100,There,0.000000e+00
2,c:1001,FriendFeed,4.750000e+07
3,c:10014,Mobclix,0.000000e+00
4,c:100265,Coastal Supply Company,0.000000e+00
...,...,...,...
9646,c:7872,Mixi,9.700000e+08
9647,c:8530,QlikTech,1.000000e+09
9648,c:9580,Pironet,4.400000e+07
9649,c:959,Zillow,2.550000e+09


In [134]:
#create df listing person id, name, company id, title, degree type, subject, institution only 
#for companies in response
df1 = ppl.merge(degs, on='object_id')
df3 = df2[['object_id','first_name','last_name','degree_type','subject','institution']]
df4 = df3.merge(relns, left_on='object_id',right_on='person_object_id')
df5 = df4[['person_object_id','first_name','last_name','relationship_object_id','title','degree_type','subject','institution']]
df5.columns=['person_id','first_name','last_name','company_id','title','degree_type','degree_subject','institution']
df6 = df5[df5['company_id'].isin(list(Y['id']))]
df6.reset_index(inplace=True, drop=True)

In [135]:
#issue: people are duplicated for each of their degrees (i.e. undergrad & mba)
X = df6[df6['person_id']=='p:266137']
X

,person_id,first_name,last_name,company_id,title,degree_type,degree_subject,institution
14884,p:266137,Ryan,E. Bateman,c:48273,Director of Sales and Trading Applications,MBA,International Business,Georgetown University
14885,p:266137,Ryan,E. Bateman,c:48273,Director of Sales and Trading Applications,BA,English Studies,Northwestern University


In [136]:
#create df listing company id, name, category, country, state, city, funding round, total funding amount only
#for companies in the response Y
dfa = objs[objs['entity_type']=='Company']
dfa = dfa[['id','name','category_code','country_code','state_code','city','funding_rounds','funding_total_usd']]
dfb = dfa[dfa['id'].isin(list(Y['id']))]
dfb.reset_index(inplace=True, drop=True)

In [137]:
dfb

,id,name,category_code,country_code,state_code,city,funding_rounds,funding_total_usd
0,c:10,Flektor,games_video,USA,CA,Culver City,0,0.0
1,c:100,There,games_video,USA,CA,San Mateo,0,0.0
2,c:1001,FriendFeed,web,USA,CA,Mountain View,1,5000000.0
3,c:10014,Mobclix,mobile,USA,CA,Palo Alto,1,0.0
4,c:100265,Coastal Supply Company,NaN,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...
9488,c:9949,PPTV,games_video,CHN,NaN,Shanghai,4,286500000.0
9489,c:99685,Symbyo Dental,biotech,NaN,NaN,NaN,1,500000.0
9490,c:997,Zimbra,enterprise,USA,CA,Palo Alto,1,14500000.0
9491,c:99737,HyperCube,public_relations,USA,TX,Lancaster,0,0.0


In [142]:
#create df with company id, funding round type, funding round amount
dfx = rounds[rounds['object_id'].isin(list(Y['id']))]
dfx = dfx[['object_id','funding_round_code','raised_amount_usd','participants']]
dfx.columns = ['id','funding_round_code','funding_round_amount','participants']
dfx.reset_index(inplace=True, drop=True)
dfx

,id,funding_round_code,funding_round_amount,participants
0,c:4,b,8500000.0,2
1,c:5,angel,500000.0,2
2,c:5,a,12700000.0,3
3,c:5,b,27500000.0,4
4,c:7299,b,10500000.0,2
...,...,...,...,...
5139,c:84079,unattributed,314960000.0,0
5140,c:62,unattributed,7755032.0,0
5141,c:73064,partial,1392000.0,0
5142,c:29130,unattributed,41750000.0,0


In [95]:
#initial visualization plots (potentially degree type, subject, institution, company industry?)